In [19]:
import pandas as pd
from textblob import TextBlob
import nltk
from textblob import Word
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,Conv1D,MaxPooling1D,GlobalMaxPooling1D,Flatten,SimpleRNN
from keras import regularizers
from sklearn.model_selection import train_test_split 
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing
from keras import layers
import numpy as np
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score

In [20]:
df=pd.read_csv("/Users/aman/coding stuff/sem 6/tarp/dataset/Suicide_preprocessed4.csv")
df=df.dropna()
df=df.iloc[0:100,:]

In [21]:
max_words = 5000
max_len = 200
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'].values)
sequences = tokenizer.texts_to_sequences(df['text'].values)
comments = pad_sequences(sequences, maxlen=max_len)

In [22]:
model3 = Sequential()
model3.add(layers.Embedding(max_words, 40, input_length=max_len))
model3.add(layers.Bidirectional(layers.LSTM(100,dropout=0.6)))
model3.add(layers.Dense(3,activation='softmax'))
model3.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
df["class"]=preprocessing.LabelEncoder().fit_transform(df["class"])
X_train,X_test,y_train,y_test=train_test_split(comments,df["class"],test_size=0.2)
y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)

In [24]:
model3.fit(X_train, y_train, epochs=5)
model3.evaluate(X_test,y_test)

Epoch 1/5
3/3 [==============================] - 3s 86ms/step - loss: 1.0719 - accuracy: 0.4375
Epoch 2/5
3/3 [==============================] - 0s 85ms/step - loss: 0.9419 - accuracy: 0.6500
Epoch 3/5
3/3 [==============================] - 0s 84ms/step - loss: 0.7904 - accuracy: 0.6500
Epoch 4/5
3/3 [==============================] - 0s 85ms/step - loss: 0.6705 - accuracy: 0.6500
Epoch 5/5
1/1 [==============================] - 0s 494ms/step - loss: 0.8959 - accuracy: 0.3500


[0.8959153890609741, 0.3499999940395355]

In [25]:
print(model3.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 200, 40)           200000    
                                                                 
 bidirectional_3 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dense_3 (Dense)             (None, 3)                 603       
                                                                 
Total params: 313,403
Trainable params: 313,403
Non-trainable params: 0
_________________________________________________________________
None


To get detail's about the models

In [28]:
# predict probabilities for test set
yhat_probs = model3.predict(X_test, verbose=0)
# predict crisp classes for test set
yhat_classes = (model3.predict(X_test) > 0.5).astype("int32")

yhat_classes=np.argmax(yhat_classes, axis=1)

# reduce to 1d array
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat_classes)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat_classes)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat_classes)
print('F1 score: %f' % f1)

# kappa
kappa = cohen_kappa_score(y_test, yhat_classes)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = roc_auc_score(y_test, yhat_probs)
print('ROC AUC: %f' % auc)
# confusion matrix
matrix = confusion_matrix(y_test, yhat_classes)
print(matrix)

1/1 [==============================] - 0s 33ms/step


ValueError: Classification metrics can't handle a mix of multilabel-indicator and binary targets